In [1]:
# DSN Doppler frequency correction for GQRX
# Requires GMAT .gmd file with time and rangerate data
# Scott Tilley, VE7TIL 

import telnetlib
import numpy as np
import time
from astropy.time import Time
from scipy.constants import c
from datetime import datetime

In [2]:
HOST = "localhost"
PORT = "7356"

tn = telnetlib.Telnet(HOST, PORT)

In [3]:
f_carrier = 8431016530.542998 # nominal carrier freqeuncy of TIANWEN-1

In [4]:
def writeCommand(cmd):
    tn.write(('%s\n' % cmd).encode('ascii'))
    return tn.read_some().decode('ascii').strip();

def writeFreq(freq):
    writeCommand("F " + str(freq))


In [5]:
freq = 8430.8e6
response = writeFreq(freq)
print(response)

None


In [6]:
gmd_file = '/home/scott/code/GMAT/R2020a/output/ve7til_2020_11_3.gmd' #uses state vector 2020-10-30T03:43:22.9642
gmd_mjd = []
gmd_rangerate = []
with open(gmd_file) as f:
    for l in f.readlines()[2:]:
        gmd_mjd.append(float(l.split()[0]))
        gmd_rangerate.append(float(l.split()[-1]))
gmd_mjd = np.array(gmd_mjd)
gmd_rangerate = np.array(gmd_rangerate)
t_gmd = Time(gmd_mjd + (2430000.0 - 2400000.5), scale = 'tai', format = 'mjd')


In [ ]:
t_data = Time(Time.now(), format = 'mjd')
i = 0  

#Find initial index
while t_gmd[i].to_value('mjd', 'long') < t_data.to_value('mjd', 'long'):
    i = i + 1
    freq_last = f_carrier * (1 - 1e3*gmd_rangerate[i]/c)
    

#loop forever and only update freqeuncy when a change is required based on rangerate data    
while True:
    t_data = Time(Time.now(), format = 'mjd')
  
    while t_gmd[i].to_value('mjd', 'long') < t_data.to_value('mjd', 'long'):  
        freq = f_carrier * (1 - 1e3*gmd_rangerate[i]/c)
        
        if int(freq) != int(freq_last):
            writeFreq(freq)
            print("Frequency {:}".format(int(freq)), end="\r")
            freq_last = freq
        i = i + 1  
    #time.sleep(0.1)      
    